### 一. 某个服务响应缓慢的排查步骤(full gc频繁, cpu占用很高, cpu和内存均占用不高时)
1. top oder by with P：1040    # 首先按进程负载排序找到  axLoad(pid)
    * -P : 按照cpu使用排序
    * -M : 按照内存使用排序

2. top -Hp 进程PID：1073       # 找到相关负载 线程PID
3. printf “0x%x\n”线程PID： 0x431   # 将线程PID转换为 16进制，为后面查找 jstack 日志做准备
    ```bash
    ➜ d printf "0x%x\n" 728             
    0x2d8
    ```
4. jstack  进程PID | vim +/十六进制线程PID -         
   例如：`jstack 1040|vim +/0x431 -`  ; 定位到代码行   
    jstack的日志往往如下: 
    ```bash
    "main" #1 prio=5 os_prio=0 tid=0x00007f8718009800 nid=0xb runnable [0x00007f871fe41000] 
       java.lang.Thread.State: RUNNABLE 
        at com.aibaobei.chapter2.eg2.UserDemo.main(UserDemo.java:9) 

    "VM Thread" os_prio=0 tid=0x00007f871806e000 nid=0xa runnable 
    ```
    * 如果定位到的线程是`VM Thread`: 表示是**gc线程导致的cpu负载过高**  
        1. 首先通过`jatst -gcutil`确定系统运行到现在执行过多少次gc  
        ```bash
        # jstat option vmid interval(ms) count, 如下图共执行了6793次full gc
        ➜  d jstat -gcutil 727 10000 10                                                                                           S0     S1     E      O      M     CCS    YGC     YGCT    FGC    FGCT     GCT
  0.00   0.00   0.00  75.07  59.09  59.60   3259    0.919  6517    7.715    8.635 
  0.00   0.00   0.00   0.08  59.09  59.60   3306    0.930  6611    7.822    8.752 
  0.00   0.00   0.00   0.08  59.09  59.60   3351    0.943  6701    7.924    8.867 
  0.00   0.00   0.00   0.08  59.09  59.60   3397    0.955  6793    8.029    8.984 
        ```
        2. 然后使用`jmap dump:format=b,file=`导出系统当前内存中的对象, 找到堆内存中哪个类的对象占比最大, 进而处理相关代码  
           若发现内存中占用最多的对象比例也不是很高, 就要看看gc日志, 是不是调用了`System.gc()`导致的频繁full gc
           ```bash
           [Full GC (System.gc()) [Tenured: 262546K->262546K(349568K), 0.0014879 secs] 262546K->262546K(506816K), [Metaspace: 3109K->3109K(1056768K)], 0.0015151 secs] [Times: user=0.00 sys=0.00, real=0.01 secs]  
[GC (Allocation Failure) [DefNew: 2795K->0K(157248K), 0.0001504 secs][Tenured: 262546K->402K(349568K), 0.0012949 secs] 265342K->402K(506816K), [Metaspace: 3109K->3109K(1056768K)], 0.0014699 secs] [Times: user=0.00
           ```
    * 如果定位到是正常的**用户线程导致cpu负载过高**, 可以通过jstack中的日志查看是哪行代码导致的CPU负载过高

    * 上面2种情况是top发现cpu占用很高或内存占用很大时,通过查找线程的方法定位. 如果top之后发现**cpu和内存都不是很高, 但是系统运行就是很缓慢**怎们办?  
      这种情况可能是3种原因导致的
        1. 接口调用比较耗时, 且不定时出现:    
           可以通过压测加大阻塞点的出现频率, 从而在出现阻塞时, 从而通过jstack查看堆栈信息找到阻塞点
        2. 如果是某个功能突然出现停滞, 且无法复线, 一般是因为锁竞争(大量线程处于waiting状态)    
           此时, 如果只打印堆栈一次, 可能发现很多线程都是WAITING或TIMED_WAITING 状态的; 因此可以每隔10秒导出一次jstack日志, 导出3,4次后, 找到这4,5个日志中都处于停滞(WAITING)的线程(一般一个线程不会停滞二三十秒), 看这些线程是阻塞在哪行代码
        3. 出现死锁  
           如果出现死锁, jstack会在底部提示出现死锁, 且告知是在哪行代码等待的锁





### 二. 关于top命令
* 关于load average: 一定程度上表示cpu负载, top分别输出1分钟,5分钟, 15分钟的平均坤负载情况  
   但是linux下负载不仅记录了running状态的进程, 还记录了sleep状态的进程; 前者占用cpu, 后者往往是因为io阻塞. 所以该数值在linux下不能100%表示cpu的负载  
* -P : 按照cpu使用排序
* -M : 按照内存使用排序

#### 1. CPU消耗分析
1. cpu主要用于中断, 内核进程, 用户进程的任务处理
2. cpu利用率
    * cpu利用率是cpu在用户进程, 内核进程, 中断处理, IO等待,空闲状态着5个人部分的使用百分比
    * linux建议用户进程的cpu消耗和内核进程的cpu消耗比率在(65%~70%)和(30%~35%)左右
    * `top`和`pidstat`用来查看cpu的消耗状况
        1. `top`  
          top命令的第三行表示CPU消耗
          * us : 用户进程所占百分比; 此部分过高, 说明某个java线程占用负载过高, 或频繁发生gc  
          * sy : 内核进程所占百分比; 此部分过高, 主要是因为:  
              &nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;java进程开启的线程比较多, 而且这些线程不断地在阻塞挂起和执行状态之间转换(等待锁, 等待IO等)
          * ni : 使用nice命令改变优先级的任务所占百分比
          * id : cpu空闲时间所占百分比
          * wa : 等待IO所占百分比
          * hi : 硬件中断所占百分比
          * si : 软件中断所占百分比
          <img src="img/top1.png">
        2. `pidstat`
           